In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

Using TensorFlow backend.
/home/randomspace/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/randomspace/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/randomspace/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/randomspace/.local/lib/python3.6/site-packag

In [2]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [3]:
data = pd.read_csv("ner_dataset.csv", encoding="latin1")
data = data.drop(['POS'], axis =1)
data = data.fillna(method="ffill")
data.head(10)

,Sentence #,Word,Tag
0,Sentence: 1,Thousands,O
1,Sentence: 1,of,O
2,Sentence: 1,demonstrators,O
3,Sentence: 1,have,O
4,Sentence: 1,marched,O
5,Sentence: 1,through,O
6,Sentence: 1,London,B-geo
7,Sentence: 1,to,O
8,Sentence: 1,protest,O
9,Sentence: 1,the,O


In [4]:
print(data.loc[data['Tag'] == 'I-art'][0:10])

         Sentence #               Word    Tag
264    Sentence: 12  Non-Proliferation  I-art
3811  Sentence: 171                V-6  I-art
4016  Sentence: 183             Simple  I-art
4017  Sentence: 183               Life  I-art
4142  Sentence: 188            Morning  I-art
4143  Sentence: 188            America  I-art
5248  Sentence: 236             Mirror  I-art
5923  Sentence: 270                 De  I-art
5924  Sentence: 270             Gaulle  I-art
5935  Sentence: 270      International  I-art


In [5]:
class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [6]:
getter = SentenceGetter(data)
sentences = getter.sentences

In [7]:
print(sentences[0])

[('Thousands', 'O'), ('of', 'O'), ('demonstrators', 'O'), ('have', 'O'), ('marched', 'O'), ('through', 'O'), ('London', 'B-geo'), ('to', 'O'), ('protest', 'O'), ('the', 'O'), ('war', 'O'), ('in', 'O'), ('Iraq', 'B-geo'), ('and', 'O'), ('demand', 'O'), ('the', 'O'), ('withdrawal', 'O'), ('of', 'O'), ('British', 'B-gpe'), ('troops', 'O'), ('from', 'O'), ('that', 'O'), ('country', 'O'), ('.', 'O')]


In [8]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words)
print(n_words)

35179


In [9]:
tags = list(set(data["Tag"].values))
n_tags = len(tags)
print(n_tags)
print(tags)

17
['I-art', 'I-per', 'I-gpe', 'I-nat', 'B-geo', 'I-tim', 'B-nat', 'O', 'I-geo', 'B-org', 'B-gpe', 'I-org', 'B-tim', 'B-art', 'I-eve', 'B-eve', 'B-per']


In [10]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [11]:
max_len = 50

X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post",value=word2idx["ENDPAD"])

y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

y = [to_categorical(i, num_classes=n_tags) for i in y]

In [12]:
print(X[0])

[ 5281 10319 14088  2711 23278 23714  6660 15830 23867 16905 15026 12710
 25331 21951 35124 16905 25732 10319 13154  1044 14225  4548   699 11813
 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178
 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178 35178
 35178 35178]


In [13]:
print(y[0])

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.

In [14]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size = 0.2)

In [15]:
print(len(X_tr))
print(len(y_tr))

38367
38367


In [16]:
print(len(X_te))
print(len(y_te))

9592
9592


In [17]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words, output_dim=max_len, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)

model = Model(input, out)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

W0812 21:57:32.560704 139841311074112 deprecation_wrapper.py:119] From /home/randomspace/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 21:57:32.562985 139841311074112 deprecation_wrapper.py:119] From /home/randomspace/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 21:57:32.573289 139841311074112 deprecation_wrapper.py:119] From /home/randomspace/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0812 21:57:32.589494 139841311074112 deprecation_wrapper.py:119] From /home/randomspace/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.p

In [18]:
# Train model
history = model.fit(X_tr,\
                    np.array(y_tr),\
                    batch_size = 32,\
                    epochs = 3,\
                    validation_split = 0.2,\
                    verbose = 1)

W0812 21:57:36.027431 139841311074112 deprecation.py:323] From /home/randomspace/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 30693 samples, validate on 7674 samples
Epoch 1/3
30693/30693 [==============================] - 166s 5ms/step - loss: 0.2085 - acc: 0.9542 - val_loss: 0.0743 - val_acc: 0.9791
Epoch 2/3
30693/30693 [==============================] - 162s 5ms/step - loss: 0.0557 - acc: 0.9838 - val_loss: 0.0525 - val_acc: 0.9845
Epoch 3/3
30693/30693 [==============================] - 164s 5ms/step - loss: 0.0394 - acc: 0.9882 - val_loss: 0.0505 - val_acc: 0.9851


In [19]:
model.summary()
model.save('model.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 50)            1758950   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 200)           120800    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 17)            3417      
Total params: 1,883,167
Trainable params: 1,883,167
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Test model
test_pred = model.predict(X_te, verbose=1)

9592/9592 [==============================] - 18s 2ms/step


In [24]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        for p in pred_i:
            p_i = np.argmax(p, axis = -1)
            out.append(idx2tag[p_i])
    return out

pred_labels = pred2label(test_pred)
true_labels = pred2label(y_te)

print(classification_report(true_labels, pred_labels))

/home/randomspace/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00        70
       B-eve       0.75      0.18      0.30        49
       B-geo       0.87      0.87      0.87      7692
       B-gpe       0.95      0.93      0.94      3125
       B-nat       0.00      0.00      0.00        42
       B-org       0.77      0.70      0.73      4019
       B-per       0.86      0.77      0.81      3295
       B-tim       0.94      0.83      0.88      4001
       I-art       0.00      0.00      0.00        54
       I-eve       0.00      0.00      0.00        45
       I-geo       0.79      0.77      0.78      1442
       I-gpe       1.00      0.29      0.45        41
       I-nat       0.00      0.00      0.00         4
       I-org       0.83      0.71      0.76      3424
       I-per       0.83      0.87      0.85      3272
       I-tim       0.87      0.67      0.75      1193
           O       0.99      1.00      1.00    447832

    accuracy              

In [25]:
# Test user's text
test_text = "When I lived in Paris last year, France was experiencing a recession. The night life was too fun, I developed an addiction to Adderall and Ritalin."

In [26]:
list_tokens = word_tokenize(test_text)
X_test = []
for token in list_tokens:
    if token not in word2idx:
        word2idx[token] = len(word2idx)
        words.append(token)
    X_test.append(word2idx[token])

X_test = pad_sequences(maxlen=max_len, sequences=[X_test], padding="post",value=word2idx["ENDPAD"])

In [28]:
p = model.predict(X_test)
p = np.argmax(p, axis=-1)
print("{:14} ({:4})".format("Word", "Pred"))
for w,pred in zip(X_test[0],p[0]):
    print("{:14}: {}".format(words[w],tags[pred]))

Word           (Pred)
When          : O
I             : O
lived         : O
in            : O
Paris         : B-geo
last          : O
year          : O
,             : O
France        : B-geo
was           : O
experiencing  : O
a             : O
recession     : O
.             : O
The           : O
night         : O
life          : O
was           : O
too           : O
fun           : O
,             : O
I             : O
developed     : O
an            : O
addiction     : O
to            : O
Adderall      : O
and           : O
Ritalin       : O
.             : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
ENDPAD        : O
